In [37]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GwiHwan-Go/Reproduce_CutMix/blob/main/Training/CutMix.ipynb)

SyntaxError: invalid syntax (1541530599.py, line 1)

In [5]:
# !pip install torch
# !pip install torchvision
# !pip install pretrainedmodels

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

from torch.utils.data import Dataset
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
import joblib
from tqdm.notebook import tqdm
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
########## YOUR DIR
import sys
sys.path.append("..") ## to import parent's folder
from Local import DIR 
########### YOUR DIR

  Using cached https://files.pythonhosted.org/packages/07/02/16044263fe6b291dfdc8bbc8103dbea8d132171bc1121f759e3d0f0e2ee0/torchvision-0.12.0-cp37-cp37m-macosx_10_9_x86_64.whl
You are using pip version 19.0.3, however version 22.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Read & Split data

In [6]:
df_train = pd.read_csv(f"{DIR}/train.csv")


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(df_train, test_size=0.2)

## Define Dataset

In [32]:
class BengaliDataset(Dataset):
    def __init__(self, csv, img_height, img_width, transform):
        self.csv = csv.reset_index()
        self.img_ids = csv['image_id'].values
        self.img_height = img_height
        self.img_width = img_width
        self.transform = transform
        
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, index):
        img_id = self.img_ids[index]
        img = joblib.load(f'{DIR}/train_images/{img_id}.pkl')
        img = img.reshape(self.img_height, self.img_width).astype(np.uint8)
        #img = 255 - img
        img = 255 - img
        #img = np.expand_dims(img, axis=2)
        
        img = img[:, :, np.newaxis]
        img = np.repeat(img, 3, 2)
        if self.transform is not None:
            img = self.transform(img)['image']
        
        label_1 = self.csv.iloc[index].grapheme_root
        label_2 = self.csv.iloc[index].vowel_diacritic
        label_3 = self.csv.iloc[index].consonant_diacritic
        
        return img, np.array([label_1, label_2, label_3])

## Define augmentations


In [29]:
import torch
import torchvision.transforms as T

train_augmentation = T.Compose([
    T.ToTensor(),
    T.RandomRotation(20),
    ##we can add more augmentation##
])

valid_augmentation = T.Compose([
    T.ToTensor(),
])

## Make Data Loader

In [33]:
from torch.utils.data import Dataset, DataLoader

train_dataset = BengaliDataset(csv=X_train,
                            img_height=137,
                            img_width=236,
                            transform=train_augmentation)
valid_dataset = BengaliDataset(csv=X_val,
                            img_height=137,
                            img_width=236,
                            transform=valid_augmentation)
train_loader = DataLoader(train_dataset,
                        shuffle=True,
                        num_workers=4,
                        batch_size=128
                       )
valid_loader = DataLoader(valid_dataset,
                       shuffle=False,
                        num_workers=4,
                        batch_size=128
                       )

## Make model, optimizer and loss function - ResNet34

In [14]:
import pretrainedmodels
model_name = 'resnet34'
model = pretrainedmodels.__dict__[model_name](pretrained='imagenet')
model

  Using cached https://files.pythonhosted.org/packages/84/0e/be6a0e58447ac16c938799d49bfb5fb7a80ac35e137547fc6cee2c08c4cf/pretrainedmodels-0.7.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/cc/ab/85d8da5c9a45e072301beb37ad7f833cd344e04c817d97e0cc75681d248f/munch-2.5.0-py2.py3-none-any.whl
  Running setup.py install for pretrainedmodels ... done
You are using pip version 19.0.3, however version 22.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
from torch.nn import Linear
## 우리 이미지 사이즈에 맞게 튜닝
in_features = model.last_linear.in_features
model.last_linear = torch.nn.Linear(in_features, 186) 

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cpu


In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                      mode='max',
                                                      verbose=True,
                                                      patience=7,
                                                      factor=0.5)

## Training

In [18]:
def rand_bbox(size, lam):
    
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [ ]:
os.environ["OMP_NUM_THREADS"]="1"  # If you are using CPU
for inputs, targets in tqdm(train_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)
    break

In [ ]:
lam = np.random.beta(1.0, 1.0) 
rand_index = torch.randperm(inputs.size()[0])

In [ ]:
targets_gra = targets[:, 0]
targets_vow = targets[:, 1]
targets_con = targets[:, 2]

In [ ]:
shuffled_targets_gra = targets_gra[rand_index]
shuffled_targets_vow = targets_vow[rand_index]
shuffled_targets_con = targets_con[rand_index]

In [ ]:
bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)

In [ ]:
inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

In [ ]:
lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))

In [ ]:
plt.imshow(inputs[5].permute(1, 2, 0).to(device)

In [ ]:
logits = model(inputs)

In [ ]:
grapheme = logits[:,:168]
vowel = logits[:, 168:179]
cons = logits[:, 179:]

In [ ]:
loss1 = loss_fn(grapheme, targets_gra) * lam + loss_fn(grapheme, shuffled_targets_gra) * (1. - lam)
loss2 = loss_fn(vowel, targets_vow) * lam + loss_fn(vowel, shuffled_targets_vow) * (1. - lam)
loss3 = loss_fn(cons, targets_con) * lam + loss_fn(cons, shuffled_targets_con) * (1. - lam)
loss = 0.5 * loss1 + 0.25 * loss2 + 0.25 * loss3

In [ ]:
for e in range(100):
    train_loss = []
    model.train()

    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        if np.random.rand() < 0.5:
            logits = model(inputs)
            grapheme = logits[:, :168]
            vowel = logits[:, 168:179]
            cons = logits[:, 179:]

            loss = loss_fn(grapheme, targets[:, 0]) + loss_fn(vowel, targets[:, 1]) + loss_fn(cons, targets[:, 2])

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            train_loss.append(loss.item())
        else:
            
            lam = np.random.beta(1.0, 1.0) 
            rand_index = torch.randperm(inputs.size()[0])
            
            targets_gra = targets[:, 0]
            targets_vow = targets[:, 1]
            targets_con = targets[:, 2]
            
            shuffled_targets_gra = targets_gra[rand_index]
            shuffled_targets_vow = targets_vow[rand_index]
            shuffled_targets_con = targets_con[rand_index]
            
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            
            logits = model(inputs)

            grapheme = logits[:,:168]
            vowel = logits[:, 168:179]
            cons = logits[:, 179:]
            
            loss1 = loss_fn(grapheme, targets_gra) * lam + loss_fn(grapheme, shuffled_targets_gra) * (1. - lam)
            loss2 = loss_fn(vowel, targets_vow) * lam + loss_fn(vowel, shuffled_targets_vow) * (1. - lam)
            loss3 = loss_fn(cons, targets_con) * lam + loss_fn(cons, shuffled_targets_con) * (1. - lam)
            
            loss = 0.5 * loss1 + 0.25 * loss2 + 0.25 * loss3
            
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            train_loss.append(loss.item())
            
            
            
            
    val_loss = []
    val_true = []
    val_pred = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in tqdm(vld_loader):
            inputs = inputs.cuda()
            targets = targets.cuda()

            logits = model(inputs)

            grapheme = logits[:, :168]
            vowel = logits[:, 168:179]
            cons = logits[:, 179:]

            loss = loss_fn(grapheme, targets[:, 0]) + loss_fn(vowel, targets[:, 1]) + loss_fn(cons, targets[:, 2])

            val_loss.append(loss.item())

            grapheme = grapheme.cpu().argmax(dim=1).data.numpy()
            vowel = vowel.cpu().argmax(dim=1).data.numpy()
            cons = cons.cpu().argmax(dim=1).data.numpy()

            val_true.append(targets.cpu().numpy())
            val_pred.append(np.stack([grapheme, vowel, cons], axis=1))

    val_true = np.concatenate(val_true)
    val_pred = np.concatenate(val_pred)

    val_loss = np.mean(val_loss)
    train_loss = np.mean(train_loss)

    score_g = recall_score(val_true[:, 0], val_pred[:, 0], average='macro')
    score_v = recall_score(val_true[:, 1], val_pred[:, 1], average='macro')
    score_c = recall_score(val_true[:, 2], val_pred[:, 2], average='macro')

    final_score = np.average([score_g, score_v, score_c], weights=[2, 1, 1])

    print(f'train_loss: {train_loss:.5f}; val_loss: {val_loss:.5f}; score: {final_score:.5f}')
    print(f'score_g: {score_g:.5f}; score_v: {score_v: .5f}, score_c: {score_c: .5f}')



    if final_score > best_score:
        best_score = final_score

        state_dict = model.cpu().state_dict()
        model = model.cuda()
        torch.save(state_dict, "model.pt")